In [4]:
import pandas as pd

# Load the data from the CSV file
df = pd.read_csv('data/bitcoin-price-filtered.csv')

# Convert the 'datetime' column to datetime objects and remove timezone info
df['datetime'] = pd.to_datetime(df['datetime']).dt.tz_localize(None)

# Create a tz-naive start date
start_date = pd.Timestamp('2019-01-01')

# Filter the DataFrame
filtered_df = df[df['datetime'] >= start_date]

# Save the filtered data
filtered_df.to_csv('bitcoin-price-filtered.csv', index=False)

In [5]:
# remove open, high, volume columns
filtered_df = filtered_df.drop(columns=['Open', 'High', 'Volume'])

# Save the filtered data
filtered_df.to_csv('bitcoin-price-filtered.csv', index=False)


In [8]:
# Convert datetime to just the date
filtered_df['date'] = filtered_df['datetime'].dt.date

# Get the last row for each day
daily_df = filtered_df.groupby('date').last().reset_index()

# Optional: sort by date if needed
daily_df = daily_df.sort_values(by='date')

# Save to a new CSV
daily_df.to_csv("data/daily-bitcoin-price.csv", index=False)

print(daily_df.head())

         date            datetime     Timestamp      Low    Close
0  2019-01-01 2019-01-01 23:59:00  1.546387e+09  3659.35  3659.81
1  2019-01-02 2019-01-02 23:59:00  1.546474e+09  3847.18  3850.00
2  2019-01-03 2019-01-03 23:59:00  1.546560e+09  3781.71  3782.09
3  2019-01-04 2019-01-04 23:59:00  1.546646e+09  3754.40  3757.66
4  2019-01-05 2019-01-05 23:59:00  1.546733e+09  3845.58  3845.58


In [9]:
# Write a script to generate a new file (bitcoin-price-change.csv) that calculates the price changes in the bitcoin price.
# Calculate daily price changes
daily_df['price_change'] = daily_df['Close'].diff()
daily_df['pct_change'] = daily_df['Close'].pct_change() * 100

# Calculate rolling 7-day and 30-day changes
daily_df['7d_change'] = daily_df['Close'].diff(periods=7)
daily_df['7d_pct_change'] = daily_df['Close'].pct_change(periods=7) * 100

daily_df['30d_change'] = daily_df['Close'].diff(periods=30) 
daily_df['30d_pct_change'] = daily_df['Close'].pct_change(periods=30) * 100

# Save price changes to new CSV
daily_df.to_csv("data/bitcoin-price-change.csv", index=False)

print("First few rows of price changes:")
print(daily_df[['date', 'Close', 'price_change', 'pct_change', '7d_change', '7d_pct_change', '30d_change', '30d_pct_change']].head())


First few rows of price changes:
         date    Close  price_change  pct_change  7d_change  7d_pct_change  \
0  2019-01-01  3659.81           NaN         NaN        NaN            NaN   
1  2019-01-02  3850.00        190.19    5.196718        NaN            NaN   
2  2019-01-03  3782.09        -67.91   -1.763896        NaN            NaN   
3  2019-01-04  3757.66        -24.43   -0.645939        NaN            NaN   
4  2019-01-05  3845.58         87.92    2.339754        NaN            NaN   

   30d_change  30d_pct_change  
0         NaN             NaN  
1         NaN             NaN  
2         NaN             NaN  
3         NaN             NaN  
4         NaN             NaN  


In [24]:
import json
import pandas as pd

def load_coin(filename, coin_name):
    # Load JSON data from the file
    with open(filename, 'r') as f:
        data = json.load(f)
    
    # Create a DataFrame only for points that have a 'c' key
    df = pd.DataFrame([
        {
            'timestamp': int(ts),
            'price': point['c'][0]
        }
        for ts, point in data['data']['points'].items()
        if 'c' in point and isinstance(point['c'], list) and len(point['c']) > 0
    ])
    
    # Convert UNIX timestamp to date (using only the date part)
    df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.date
    
    # Group by date and get the last record of the day
    df_daily = df.groupby('date', as_index=False).last()[['date', 'price']]
    
    # Rename the 'price' column to the coin's name
    df_daily = df_daily.rename(columns={'price': coin_name})
    return df_daily

# Load each coin's data
bonk_df = load_coin('data/meme-coins/bonk.json', 'bonk')
doge_df = load_coin('data/meme-coins/doge.json', 'doge')
shib_df = load_coin('data/meme-coins/shib.json', 'shib')
pepe_df = load_coin('data/meme-coins/pepe.json', 'pepe')
floki_df = load_coin('data/meme-coins/floki.json', 'floki')

# Merge all coin DataFrames on the date column
merged_df = bonk_df.merge(doge_df, on='date', how='outer') \
                   .merge(shib_df, on='date', how='outer') \
                   .merge(pepe_df, on='date', how='outer') \
                   .merge(floki_df, on='date', how='outer')

# Sort by date
merged_df = merged_df.sort_values('date')

# Calculate day-over-day percent change for each coin column
coin_cols = [col for col in merged_df.columns if col != 'date']
pct_change_df = merged_df[coin_cols].pct_change() * 100

# Calculate average percent change for all coins (ignoring NaN values for the first day)
merged_df['avg_pct_change'] = pct_change_df.mean(axis=1)

# Save the simplified CSV with the average percent change column
merged_df.to_csv('data/meme-coins/simplified_prices.csv', index=False)

print("Simplified CSV created with columns:", merged_df.columns.tolist())

Simplified CSV created with columns: ['date', 'bonk', 'doge', 'shib', 'pepe', 'floki', 'avg_pct_change']


/var/folders/fm/9lqxl2_d1n74fpfks5v25c300000gn/T/ipykernel_66127/1781707447.py:47: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  pct_change_df = merged_df[coin_cols].pct_change() * 100


In [ ]:
# Review anomolies in the data
print(merged_df[merged_df['avg_pct_change'].isna()])

# Remove rows with NaN values
merged_df = merged_df.dropna(subset=['avg_pct_change'])



In [25]:
# add seven day avg percent change column to simplified_prices.csv
merged_df['7d_avg_pct_change'] = merged_df['avg_pct_change'].rolling(window=7).mean()

# save the updated dataframe
merged_df.to_csv('data/meme-coins/simplified_prices.csv', index=False)

